In [10]:
from network import get_sts_model
from triplet_dataset import TripletDataset, get_dataset
# model_path = 'ammarnasr/LoRa_all-MiniLM-L12-v1'
# model = get_sts_model(model_path)
# tokenizer = model.tokenizer
data_df = get_dataset()

Creating triplets:   0%|          | 0/14 [00:00<?, ?group/s]

In [12]:
# Split dataframes into train, validation, and test sets (80%, 10%, 10%)
train_df = data_df.sample(frac=0.8, random_state=42)
test_df = data_df.drop(train_df.index)
val_df = test_df.sample(frac=0.5, random_state=42)
test_df = test_df.drop(val_df.index)



In [17]:
val_df

,triplet,positive_group,negative_group
417,"<InputExample> label: 0, texts: تحياتي; ازيكم;...",1.0,6.0
5423,"<InputExample> label: 0, texts: بسأل من الودائ...",11.0,13.0
3146,"<InputExample> label: 0, texts: كيف انشي حساب ...",7.0,2.0
302,"<InputExample> label: 0, texts: هلا; مساء السع...",1.0,8.0
2386,"<InputExample> label: 0, texts: وكم أقصي مبلغ ...",5.0,14.0
...,...,...,...
3419,"<InputExample> label: 0, texts: سلام عليكم هل ...",7.0,4.0
4496,"<InputExample> label: 0, texts: هل يوجد لديكم ...",9.0,10.0
6794,"<InputExample> label: 0, texts: بسأل من التموي...",14.0,11.0
2327,"<InputExample> label: 0, texts: ممكن اشتري ريا...",5.0,4.0


In [28]:
triplets[1][0]

'بسأل من الودائع الاستثمارية'

In [ ]:
def triplets_df_to_single_ds(triplets_df):
    triplets = triplets_df['triplet'].tolist()
    triplets = [triplet.texts for triplet in triplets]
    positive_groups = triplets_df['positive_group'].tolist()
    negative_groups = triplets_df['negative_group'].tolist()
    sentences =[]
    groups = []
    for triplet, positive_group, negative_group in zip(triplets, positive_groups, negative_groups):
        anchor, positive, negative = triplet
        if anchor not in sentences:
            sentences.append(anchor)
            groups.append(positive_group)
        if positive not in sentences:
            sentences.append(positive)
            groups.append(positive_group)
        if negative not in sentences:
            sentences.append(negative)
            groups.append(negative_group)
    data = {'sentence': sentences, 'group': groups}
    df = pd.DataFrame(data)
    return df

In [34]:
import pandas as pd
triplets = val_df['triplet'].tolist()
triplets = [triplet.texts for triplet in triplets]
positive_groups = val_df['positive_group'].tolist()
negative_groups = val_df['negative_group'].tolist()
sentences = []
groups = []
for triplet, positive_group, negative_group in zip(triplets, positive_groups, negative_groups):
    anchor, positive, negative = triplet
    if anchor not in sentences:
        sentences.append(anchor)
        groups.append(positive_group)
    if positive not in sentences:
        sentences.append(positive)
        groups.append(positive_group)
    if negative not in sentences:
        sentences.append(negative)
        groups.append(negative_group)
data = {'sentence': sentences, 'group': groups}
df = pd.DataFrame(data)
df


,sentence,group
0,تحياتي,1.0
1,ازيكم,1.0
2,مطلوبات حساب الدولار,6.0
3,بسأل من الودائع الاستثمارية,11.0
4,بسأل من الودائع الاستثمارية شروطها و ارباحها و...,11.0
...,...,...
247,دايرة غرض للدراسة خارج السودان,14.0
248,ارجو منكم افادتي باسعر الريال السعودي,4.0
249,عاوز اعرف اسعار العملات,4.0
250,داير اتكلم مع زول من خدمة العملا,3.0


In [30]:
group_sentence_dict

{1.0: ['تحياتي',
  'ازيكم',
  'هلا',
  'مساء السعاده عليكم',
  'يا ناس بنك الاهلى',
  'مرحبا',
  'اهلين',
  'رمضان مبارك عليكم',
  'اهلا',
  'يا عالم',
  'هوي ها',
  'السلام عليكم ورحمة الله',
  'مرحب',
  'مساء الخير',
  'الساده البنك الأهلي السوداني',
  'السلام عليكم',
  'الو',
  'سلام',
  'يا ناس',
  'صبأح ألورود',
  'السلام عليكم ورحمة الله وبركاتة',
  'الوووو',
  'سلامات',
  'هاي'],
 6.0: ['مطلوبات حساب الدولار',
  'داير افتح حساب ادخار ما هى المتطلبات والشروط لفتح الحساب',
  'حساب الدولار بفتحو كيف',
  'داير أفتح حساب بالعملة الأجنبية ',
  'داير افتح حساب ادخار',
  'وكيف افتح حساب بنكي',
  'شنو المطلوب عشان افتح حساب بالنقد الاجنبي',
  'خطوات فتح الحساب ',
  'يوجد فتح حساب في الفروع',
  'و خلال كم يوم بعد افتح حساب يستلم بطاقة الصراف و اشغل تطبيق البنك الأهلي السوداني',
  'وكم من المبلغ يتم إيداعه لفتح الحساب',
  'ممكن افتح حساب',
  'أود فتح حساب جاري لدي البنك الأهلي السوداني',
  'داير افتح عندكم حساب المطلوب شنو',
  'المطلوووب شو لقفتح حساااب',
  'وطريقة فتح حساب ادخار',
  'عايز

In [6]:
train_dataset = TripletDataset(None, tokenizer=tokenizer, use_allnli=True, eval_data=True)
sentences = train_dataset.data
sample_batch = next(iter(train_dataset))


Batching data:   0%|          | 0/1000 [00:00<?, ?row/s]

Tokenizing data:   0%|          | 0/101 [00:00<?, ?batch/s]

In [8]:
sentences= [i.texts[0] for i in sentences]


In [9]:
sentences

['A girl is inspecting a machine.',
 'A woman displays the goods for sale in the bakery.',
 'Two elderly people are wearing blue shirts.',
 'A man and woman stand near a large red sculpture outdoors.',
 'A young man looks at a coffee marker on the shelf of a department store.',
 "There's a man taking photos of a structure",
 'There are women sitting down.',
 'a man on a suit walking down a very colorful street',
 'Asian woman harvesting wheat.',
 'Some people are swimming in water; one has goggles on.',
 'A squirrel with a fluffy tail sits on a tree branch among yellow leaves.',
 'A person wearing a blue robe is in the air, holding a sword.',
 'Two young males weaving fishing nets.',
 'There are animals performing.',
 'Two people are flying on a carpet.',
 'A little boy playing with his toy.',
 'Whales leap into the air.',
 'A man in a suit and with a name tag and red tie stands in a dining area.',
 'A baby cries on a blanket',
 'This family is relaxing outside of their motorhome.',
 '

In [ ]:
import modeltraining 
from peft import LoraConfig
rank = 64
peft_config = LoraConfig(inference_mode=False,
            r=rank,
            lora_alpha=rank*2,
            lora_dropout=0.05,
            target_modules=['value','query','key', 'dense']
            )

training_args = {'use_allnli': True, 'peft_config': peft_config}
modeltraining.main(training_args)